In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
data = pd.read_csv("../input/movie_metadata.csv")

In [5]:
data.head()

In [6]:
data.describe()

In [7]:
data['imdb_score'].describe()

In [8]:
data.columns

In [9]:
data['imdb_score'].hist(bins=30)

In [10]:
corr = data.select_dtypes(include = ['float64','int64']).iloc[:,1:].corr()
plt.figure(figsize=(7, 7))
sns.heatmap(corr,vmax=1,square= True)

In [12]:
sns.regplot(x='num_voted_users',y='imdb_score',data=data)


In [13]:
data.info()

In [14]:
plt.figure(figsize=(1000,1000))
sns.pairplot(data,x_vars=['num_voted_users','num_user_for_reviews','duration','movie_facebook_likes','title_year','gross','director_facebook_likes','cast_total_facebook_likes','facenumber_in_poster','actor_1_facebook_likes','actor_2_facebook_likes','actor_3_facebook_likes'],y_vars=['imdb_score'])

In [17]:
plt.figure(figsize=(50,30))
sns.stripplot(x=data['language'],y=data['imdb_score'])

In [18]:
data2=data[['language','imdb_score']]
data2=data2.dropna()

In [19]:
plt.figure(figsize=(12,6))
sns.countplot(x='language',data=data2)


In [20]:
plt.figure(figsize=(12,6))
sns.countplot(x='content_rating',data=data)

In [21]:
plt.figure(figsize=(100,30))
sns.countplot(x='country',data=data)

In [22]:
plt.figure(figsize=(12,6))
sns.boxplot(x='content_rating',y='imdb_score',data=data)


In [23]:
plt.figure(figsize=(20,6))
sns.boxplot(x='language',y='imdb_score',data=data)

In [24]:
data.head()

model exploration

In [25]:
y = data['imdb_score']


In [26]:
x = data.drop('imdb_score',1)

In [27]:
x.shape

In [28]:
#check missing value 
missing_data = data.isnull().sum(axis=0).reset_index()
missing_data.columns=['column_name','missing_count']
missing_data = missing_data.loc[missing_data['missing_count']>0]
missing_data = missing_data.sort_values(by='missing_count')

ind = np.arange(missing_data.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_data.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_data.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

missing value

In [29]:
#fill missing value of budget
budgetmean=data['budget'].mean()
data['budget']=data['budget'].fillna(budgetmean)
#fill missing value of aspect
aspect=data['aspect_ratio'].mean()
data['aspect_ratio']=data['aspect_ratio'].fillna(aspect)
#fill missing value of num_user_for_reviews
num_user_review = data['num_user_for_reviews'].mean()
data['num_user_for_reviews']=data['num_user_for_reviews'].fillna(num_user_review)
#fill missing value of duration
duration = data['duration'].mean()
data['duration']=data['duration'].fillna(num_user_review)#fill missing value of budget
#fill missing value of num_critic_for_reviews
num_critic_for_reviews = data['num_critic_for_reviews'].mean()
data['num_critic_for_reviews']=data['num_critic_for_reviews'].fillna(num_critic_for_reviews)
data.head()


In [30]:
y.shape

In [31]:
x = data[['actor_3_facebook_likes', 'actor_1_facebook_likes', 'gross',
       'num_voted_users', 'cast_total_facebook_likes', 'facenumber_in_poster',
       'num_user_for_reviews', 'budget', 'title_year',
       'actor_2_facebook_likes', 'aspect_ratio',
       'movie_facebook_likes']].fillna(0)

In [32]:
y = data['imdb_score']

In [33]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=101)

In [34]:
from sklearn.linear_model import LinearRegression

In [35]:
x_train.head()

In [36]:
regression = LinearRegression()


In [37]:
regression.fit(x_train,y_train)

In [38]:
pred=regression.predict(x_test)

In [39]:
error=y_test - pred
sns.distplot(error,bins=50);

evaluate model

In [40]:
plt.scatter(y_test,pred)
plt.xlabel('true value')
plt.ylabel('prediction value')

In [41]:
coeffecients = pd.DataFrame(regression.coef_,x.columns)
coeffecients.columns = ['Coeffecient']
coeffecients